In [1]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

## Load train dataset

In [3]:
BASE_DATASET = "../../data/mlspec-blackfriday/dataset/raw"
os.makedirs(BASE_DATASET, exist_ok=True)

In [4]:
!gsutil cp -r gs://mlteam-ml-specialization-2021-blackfriday/dataset/raw/* $BASE_DATASET/

Copying gs://mlteam-ml-specialization-2021-blackfriday/dataset/raw/README.md...
Copying gs://mlteam-ml-specialization-2021-blackfriday/dataset/raw/test.csv...
Copying gs://mlteam-ml-specialization-2021-blackfriday/dataset/raw/train.csv...

Operation completed over 3 objects/33.5 MiB.                                     


In [5]:
TRAIN_SET = os.path.join(BASE_DATASET, 'train.csv')
df = pd.read_csv(TRAIN_SET)

In [6]:
df.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

## Define functions and columns to group on

In [7]:
def get_grouped_df(df, COLUMNS, aggregation_dict):
    df_totals = df.groupby(COLUMNS).agg(aggregation_dict)
    
    df_totals.columns = ['_'.join(col) for col in df_totals.columns.values]
    df_totals = df_totals.reset_index()

    return df_totals

In [8]:
PRODUCT_COLUMNS = [
    'Product_ID', 'Product_Category_1',
    'Product_Category_2', 'Product_Category_3'
]

aggregation_dict = {
    'User_ID': 'count',
    'Purchase': ['count', 'sum', 'min', 'mean', 'max', 'std']
}

## Get overall ranking

In [9]:
df_totals = get_grouped_df(df, PRODUCT_COLUMNS, aggregation_dict)
df_totals['rank'] = df_totals['Purchase_count'].rank(method='min', ascending=False)

In [13]:
df_totals.sort_values('Purchase_count', ascending=False).head()

,Product_ID,Product_Category_1,Product_Category_2,Product_Category_3,User_ID_count,Purchase_count,Purchase_sum,Purchase_min,Purchase_mean,Purchase_max,Purchase_std,rank
39,P00025442,1,2.0,9.0,1615,1615,27995166,3961,17334.468111,19707,2955.609692,1.0
148,P00110742,1,2.0,8.0,1612,1612,26722309,3798,16577.114764,19708,3266.793787,2.0
152,P00112142,1,2.0,14.0,1562,1562,24216006,3793,15503.204866,19706,3574.019615,3.0
88,P00057642,1,15.0,16.0,1470,1470,23102780,3890,15716.176871,19708,3470.043730,4.0
287,P00184942,1,8.0,17.0,1440,1440,24334887,3809,16899.227083,19707,3142.245422,5.0


# Age related top k

## Age 0-17 vs. overall

In [19]:
df_age_0_17 = df.loc[df['Age'] == '0-17']

# get ranking in subset
df_totals_age_0_17 = get_grouped_df(df_age_0_17, PRODUCT_COLUMNS, aggregation_dict)
df_totals_age_0_17['rank'] = df_totals_age_0_17['Purchase_count'].rank(method='min', ascending=False)

In [21]:
pd.merge(
    df_totals[['Product_ID','rank']], df_totals_age_0_17[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_total', '_age_0_17'),
    how='outer'
).nsmallest(10, 'rank_age_0_17', keep='all')

,Product_ID,rank_total,rank_age_0_17
214,P00145042,7.0,1.0
152,P00112142,3.0,2.0
53,P00034742,14.0,3.0
383,P00242742,13.0,3.0
0,P00000142,18.0,5.0
374,P00237542,9.0,6.0
39,P00025442,1.0,7.0
8,P00003442,25.0,8.0
431,P00289942,35.0,8.0
148,P00110742,2.0,10.0


## Age 18-25 vs. overall

In [22]:
df_age_18_25 = df.loc[df['Age'] == '18-25']

# get ranking in subset
df_totals_age_18_25 = get_grouped_df(df_age_18_25, PRODUCT_COLUMNS, aggregation_dict)
df_totals_age_18_25['rank'] = df_totals_age_18_25['Purchase_count'].rank(method='min', ascending=False)

In [57]:
pd.merge(
    df_totals[['Product_ID','rank']], df_totals_age_18_25[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_total', '_age_18_25'),
    how='outer'
).nsmallest(10, 'rank_age_18_25', keep='all')

,Product_ID,rank_total,rank_age_18_25
152,P00112142,3.0,1.0
148,P00110742,2.0,2.0
374,P00237542,9.0,3.0
72,P00046742,6.0,4.0
383,P00242742,13.0,5.0
16,P00010742,10.0,6.0
88,P00057642,4.0,6.0
214,P00145042,7.0,6.0
39,P00025442,1.0,9.0
149,P00110842,11.0,10.0


## Age 0-17 vs age 18-25

In [24]:
pd.merge(
    df_totals_age_0_17[['Product_ID','rank']], df_totals_age_18_25[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_age_0_17', '_age_18_25'),
    how='outer'
).nsmallest(10, 'rank_age_18_25', keep='all')

,Product_ID,rank_age_0_17,rank_age_18_25
135,P00112142,2.0,1.0
131,P00110742,10.0,2.0
336,P00237542,6.0,3.0
64,P00046742,19.0,4.0
341,P00242742,3.0,5.0
15,P00010742,36.0,6.0
79,P00057642,24.0,6.0
193,P00145042,1.0,6.0
37,P00025442,7.0,9.0
132,P00110842,17.0,10.0


In [25]:
pd.merge(
    df_totals_age_0_17[['Product_ID','rank']], df_totals_age_18_25[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_age_0_17', '_age_18_25'),
    how='outer'
).nsmallest(10, 'rank_age_0_17', keep='all')

,Product_ID,rank_age_0_17,rank_age_18_25
193,P00145042,1.0,6.0
135,P00112142,2.0,1.0
50,P00034742,3.0,14.0
341,P00242742,3.0,5.0
0,P00000142,5.0,12.0
336,P00237542,6.0,3.0
37,P00025442,7.0,9.0
8,P00003442,8.0,49.0
380,P00289942,8.0,35.0
131,P00110742,10.0,2.0


## Age 0-17 vs age 55+

In [58]:
df_age_55_plus = df.loc[df['Age'] == '55+']

# get ranking in subset
df_totals_age_55_plus = get_grouped_df(df_age_55_plus, PRODUCT_COLUMNS, aggregation_dict)
df_totals_age_55_plus['rank'] = df_totals_age_55_plus['Purchase_count'].rank(method='min', ascending=False)

In [59]:
pd.merge(
    df_totals_age_0_17[['Product_ID','rank']], df_totals_age_55_plus[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_age_0_17', '_age_55_plus'),
    how='outer'
).nsmallest(10, 'rank_age_0_17', keep='all')

,Product_ID,rank_age_0_17,rank_age_55_plus
193,P00145042,1.0,19.0
135,P00112142,2.0,17.0
50,P00034742,3.0,11.0
341,P00242742,3.0,35.0
0,P00000142,5.0,59.0
336,P00237542,6.0,48.0
37,P00025442,7.0,2.0
8,P00003442,8.0,29.0
380,P00289942,8.0,59.0
131,P00110742,10.0,9.0


# City_Category related top k

## City A vs overall

In [36]:
df_City_Category_A = df.loc[df['City_Category'] == 'A']

# get ranking in subset
df_totals_City_Category_A = get_grouped_df(df_City_Category_A, PRODUCT_COLUMNS, aggregation_dict)
df_totals_City_Category_A['rank'] = df_totals_City_Category_A['Purchase_count'].rank(method='min', ascending=False)

In [37]:
pd.merge(
    df_totals[['Product_ID','rank']], df_totals_City_Category_A[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_total', '_City_Category_A'),
    how='outer'
).nsmallest(10, 'rank_City_Category_A', keep='all')

,Product_ID,rank_total,rank_City_Category_A
148,P00110742,2.0,1.0
39,P00025442,1.0,2.0
88,P00057642,4.0,3.0
94,P00059442,7.0,4.0
152,P00112142,3.0,5.0
214,P00145042,7.0,6.0
149,P00110842,11.0,7.0
374,P00237542,9.0,8.0
72,P00046742,6.0,9.0
161,P00114942,17.0,10.0


## City B vs overall

In [38]:
df_City_Category_B = df.loc[df['City_Category'] == 'B']

# get ranking in subset
df_totals_City_Category_B = get_grouped_df(df_City_Category_B, PRODUCT_COLUMNS, aggregation_dict)
df_totals_City_Category_B['rank'] = df_totals_City_Category_B['Purchase_count'].rank(method='min', ascending=False)

In [39]:
pd.merge(
    df_totals[['Product_ID','rank']], df_totals_City_Category_A[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_total', '_City_Category_B'),
    how='outer'
).nsmallest(10, 'rank_City_Category_B', keep='all')

,Product_ID,rank_total,rank_City_Category_B
148,P00110742,2.0,1.0
39,P00025442,1.0,2.0
88,P00057642,4.0,3.0
94,P00059442,7.0,4.0
152,P00112142,3.0,5.0
214,P00145042,7.0,6.0
149,P00110842,11.0,7.0
374,P00237542,9.0,8.0
72,P00046742,6.0,9.0
161,P00114942,17.0,10.0


## City C vs. overall

In [40]:
df_City_Category_C = df.loc[df['City_Category'] == 'C']

# get ranking in subset
df_totals_City_Category_C = get_grouped_df(df_City_Category_C, PRODUCT_COLUMNS, aggregation_dict)
df_totals_City_Category_C['rank'] = df_totals_City_Category_C['Purchase_count'].rank(method='min', ascending=False)

In [41]:
pd.merge(
    df_totals[['Product_ID','rank']], df_totals_City_Category_C[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_total', '_City_Category_C'),
    how='outer'
).nsmallest(10, 'rank_City_Category_C', keep='all')

,Product_ID,rank_total,rank_City_Category_C
39,P00025442,1.0,1.0
152,P00112142,3.0,2.0
148,P00110742,2.0,3.0
287,P00184942,5.0,4.0
88,P00057642,4.0,5.0
72,P00046742,6.0,6.0
16,P00010742,10.0,7.0
374,P00237542,9.0,8.0
94,P00059442,7.0,9.0
214,P00145042,7.0,10.0


# Marital status top k

## Marital status 0

In [45]:
df_Marital_Status_0 = df.loc[df['Marital_Status'] == 0]

# get ranking in subset
df_totals_Marital_Status_0 = get_grouped_df(df_Marital_Status_0, PRODUCT_COLUMNS, aggregation_dict)
df_totals_Marital_Status_0['rank'] = df_totals_Marital_Status_0['Purchase_count'].rank(method='min', ascending=False)

In [46]:
pd.merge(
    df_totals[['Product_ID','rank']], df_totals_Marital_Status_0[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_total', '_Marital_Status_0'),
    how='outer'
).nsmallest(10, 'rank_Marital_Status_0', keep='all')

,Product_ID,rank_total,rank_Marital_Status_0
148,P00110742,2.0,1.0
39,P00025442,1.0,2.0
152,P00112142,3.0,3.0
88,P00057642,4.0,4.0
214,P00145042,7.0,5.0
72,P00046742,6.0,6.0
374,P00237542,9.0,7.0
287,P00184942,5.0,8.0
94,P00059442,7.0,9.0
16,P00010742,10.0,10.0


## Marital Status 1

In [47]:
df_Marital_Status_1 = df.loc[df['Marital_Status'] == 1]

# get ranking in subset
df_totals_Marital_Status_1 = get_grouped_df(df_Marital_Status_1, PRODUCT_COLUMNS, aggregation_dict)
df_totals_Marital_Status_1['rank'] = df_totals_Marital_Status_1['Purchase_count'].rank(method='min', ascending=False)

In [48]:
pd.merge(
    df_totals[['Product_ID','rank']], df_totals_Marital_Status_1[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_total', '_Marital_Status_1'),
    how='outer'
).nsmallest(10, 'rank_Marital_Status_1', keep='all')

,Product_ID,rank_total,rank_Marital_Status_1
39,P00025442,1.0,1.0
148,P00110742,2.0,2.0
152,P00112142,3.0,3.0
287,P00184942,5.0,4.0
88,P00057642,4.0,5.0
94,P00059442,7.0,6.0
16,P00010742,10.0,7.0
72,P00046742,6.0,8.0
374,P00237542,9.0,9.0
214,P00145042,7.0,10.0


# Gender top k

## Male vs Female

In [50]:
df_Gender_M = df.loc[df['Gender'] == 'M']

# get ranking in subset
df_totals_Gender_M = get_grouped_df(df_Gender_M, PRODUCT_COLUMNS, aggregation_dict)
df_totals_Gender_M['rank'] = df_totals_Gender_M['Purchase_count'].rank(method='min', ascending=False)

In [52]:
df_Gender_F = df.loc[df['Gender'] == 'F']

# get ranking in subset
df_totals_Gender_F = get_grouped_df(df_Gender_F, PRODUCT_COLUMNS, aggregation_dict)
df_totals_Gender_F['rank'] = df_totals_Gender_F['Purchase_count'].rank(method='min', ascending=False)

In [54]:
pd.merge(
    df_totals_Gender_F[['Product_ID','rank']], df_totals_Gender_M[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_Gender_F', '_Gender_M'),
    how='outer'
).nsmallest(10, 'rank_Gender_M', keep='all')

,Product_ID,rank_Gender_F,rank_Gender_M
38,P00025442,5.0,1.0
145,P00110742,1.0,2.0
149,P00112142,7.0,3.0
87,P00057642,20.0,4.0
283,P00184942,13.0,5.0
71,P00046742,11.0,6.0
370,P00237542,15.0,7.0
211,P00145042,12.0,8.0
15,P00010742,16.0,9.0
92,P00059442,3.0,10.0


In [55]:
pd.merge(
    df_totals_Gender_F[['Product_ID','rank']], df_totals_Gender_M[['Product_ID','rank']],
    on=['Product_ID'], suffixes=('_Gender_F', '_Gender_M'),
    how='outer'
).nsmallest(10, 'rank_Gender_F', keep='all')

,Product_ID,rank_Gender_F,rank_Gender_M
145,P00110742,1.0,2.0
52,P00034742,2.0,20.0
92,P00059442,3.0,10.0
146,P00110842,4.0,15.0
38,P00025442,5.0,1.0
0,P00000142,6.0,21.0
149,P00112142,7.0,3.0
137,P00102642,8.0,16.0
8,P00003442,9.0,34.0
221,P00148642,10.0,18.0
